# MARATONA BEHIND THE CODE 2020

## DESAFIO 2: PARTE 2

### Introdução

Na parte 1 deste desafio, você realizou o pré-processamento e o treinamento de um modelo a partir de um conjunto de dados base fornecido. Nesta segunda etapa você irá integrar todas as transformações e eventos de treinamento criados anteriormente em uma Pipeline completa para *deploy* no **Watson Machine Learning**!

<hr>

### Preparação do Notebook

Primeiro realizaremos a instalação do scikit-learn e a importação das mesmas bibliotecas utilizadas anteriormente

In [1]:
# Primeiro, realizamos a instalação do scikit-learn versão 0.20.3 e do xgboost versão 0.71 no Kernel deste notebook
# ** CUIDADO AO TROCAR A VERSÃO DAS BIBLIOTECAS -- VERSÕES DIFERENTES PODEM SER INCOMPATÍVEIS COM O WATSON STUDIO **
# OBS: A instalação do xgboost leva um tempo considerável
!pip install scikit-learn==0.20.3 --upgrade
!pip install xgboost==0.71 --upgrade

Defaulting to user installation because normal site-packages is not writeable
     |████▊                           | 788 kB 183 kB/s eta 0:00:25^C

ERROR: Operation cancelled by user
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 494 kB 282 kB/s eta 0:00:01
^C
ERROR: Operation cancelled by user


In [1]:
import json
import requests
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate

É necessário inserir o conjunto de dados base novamente como um dataframe pandas, seguindo as instruções

![alt text](https://i.imgur.com/K1DwL9I.png "importing-csv-as-df")

Após a seleção da opção **"Insert to code"**, a célula abaixo será preenchida com o código necessário para importação e leitura dos dados no arquivo .csv como um DataFrame Pandas.

In [ ]:

<< INSIRA O DATASET COMO UM PANDAS DATAFRAME NESTA CÉLULA! >>>


In [2]:
df_data_1 = pd.read_csv("../data_asset/dataset_desafio_2.csv")

### Construção da Pipeline completa para encapsulamento no WML

#### Preparando transformações personalizadas para carregamento no WML

Na etapa anterior, foi mostrado como criar uma transformação personalizada, através da declaração de uma classe Python com os métodos ``fit`` e ``transform``.

    - Código da transformação personalizada DropColumns():
    
    from sklearn.base import BaseEstimator, TransformerMixin
    # All sklearn Transforms must have the `transform` and `fit` methods
    class DropColumns(BaseEstimator, TransformerMixin):
        def __init__(self, columns):
            self.columns = columns
        def fit(self, X, y=None):
            return self
        def transform(self, X):
            # Primeiro realizamos a cópia do dataframe 'X' de entrada
            data = X.copy()
            # Retornamos um novo dataframe sem as colunas indesejadas
            return data.drop(labels=self.columns, axis='columns')

Para integrar esses tipos de transformações personalizadas nas Pipelines do Watson Machine Learning, é necessário primeiramente empacotar seu código personalizado como uma biblioteca Python. Isso pode ser feito facilmente com o uso da ferramenta *setuptools*.

No seguinte repositório git: https://github.com/vnderlev/sklearn_transforms temos todos os arquivos necessários para a criação de um pacote Python, nomeado **my_custom_sklearn_transforms**.
Esse pacote possui a seguinte estrutura de arquivos:

    /my_custom_sklearn_transforms.egg-info
        dependency_links.txt
        not-zip-safe
        PKG-INFO
        SOURCES.txt
        top_level.txt
    /my_custom_sklearn_transforms
        __init__.py
        sklearn_transformers.py
    PKG-INFO
    README.md
    setup.cfg
    setup.py
    
O arquivo principal, que irá conter o código das nossas transformadas personalizadas, é o arquivo **/my_custom_sklearn_transforms/sklearn_transformers.py**. Se você acessá-lo no repositório, irá notar que ele contém exatamente o mesmo código declarado na primeira etapa (a classe DropColumns).

Caso você tenha declarado transformações próprias (além da DropColumn fornecida), você deverá adicionar todas as classes dessas transformadas criadas por você nesse mesmo arquivo. Para tal, você deve realizar o fork desse repositório (isso pode ser feito na própria interface Web do Github, clicando no botão conforme a imagem abaixo), e adicionar suas classes personalizadas no arquivo **sklearn_transformers.py**.

![alt text](https://i.imgur.com/D81E1uM.png "forking-a-repo")

Se você somente fez o uso da transformação fornecida (DropColumns), pode ignorar essa etapa de fork, e seguir utilizando o pacote base fornecido! :)

Após a preparação do seu pacote Python com as suas transformadas personalizadas, substitua o link do repositório git na célula abaixo e execute-a. Caso você não tenha preparado nenhuma nova transformada, execute a célula com o link do repositório já fornecido. 

<hr>
    
**OBSERVAÇÃO**

Caso a execução da célula abaixo retorne um erro de que o repositório já existe, execute:

**!rm -r -f sklearn_transforms**

In [31]:
# substitua o link abaixo pelo link do seu repositório git (se for o caso)
!git clone https://github.com/joel021/create_model_to_pipeline.git

Cloning into 'create_model_to_pipeline'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 146 (delta 18), reused 19 (delta 10), pack-reused 115
Receiving objects: 100% (146/146), 22.06 KiB | 1.84 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [32]:
!cd create_model_to_pipeline
!ls -ltr

total 240
-rw-r--r-- 1 joel joel  27701 ago 23 11:49 notebook:parte-1_XTm42MgVg.ipynb
-rw-r--r-- 1 joel joel 210830 ago 23 15:31 notebook:parte-2_XvWXXx6YQ.ipynb
drwxr-xr-x 6 joel joel   4096 ago 23 15:32 create_model_to_pipeline


Para subir o código no WML, precisamos enviar um arquivo .zip com todo o código fonte, então iremos zipar o diretório clonado em seguida:

In [33]:
!zip -r create_model_to_pipeline.zip create_model_to_pipeline

  adding: create_model_to_pipeline/ (stored 0%)
  adding: create_model_to_pipeline/PKG-INFO (deflated 31%)
  adding: create_model_to_pipeline/my_custom_model.egg-info/ (stored 0%)
  adding: create_model_to_pipeline/my_custom_model.egg-info/dependency_links.txt (stored 0%)
  adding: create_model_to_pipeline/my_custom_model.egg-info/SOURCES.txt (deflated 48%)
  adding: create_model_to_pipeline/my_custom_model.egg-info/PKG-INFO (deflated 32%)
  adding: create_model_to_pipeline/my_custom_model.egg-info/not-zip-safe (stored 0%)
  adding: create_model_to_pipeline/my_custom_model.egg-info/top_level.txt (stored 0%)
  adding: create_model_to_pipeline/.git/ (stored 0%)
  adding: create_model_to_pipeline/.git/branches/ (stored 0%)
  adding: create_model_to_pipeline/.git/index (deflated 47%)
  adding: create_model_to_pipeline/.git/hooks/ (stored 0%)
  adding: create_model_to_pipeline/.git/hooks/pre-commit.sample (deflated 43%)
  adding: create_model_to_pipeline/.git/hooks/prepare-commit-msg.sample

Com o arquivo zip do nosso pacote carregado no Kernel deste notebook, podemos utilizar a ferramenta pip para instalá-lo, conforme a célula abaixo:

In [34]:
!pip install create_model_to_pipeline.zip

Defaulting to user installation because normal site-packages is not writeable
Processing ./create_model_to_pipeline.zip
  Created wheel for dense-model-to-pipeline: filename=dense_model_to_pipeline-1.0-py3-none-any.whl size=4512 sha256=135cbd2d1a9af4d805216a9cb8ed7cd3b13ad8ba5cfbb029488f95fbd9805f78
  Stored in directory: /tmp/pip-ephem-wheel-cache-fu_0k4v7/wheels/3c/54/1d/84938028f909b939605b09d22dfd467ea462ecbded855f4eeb
Successfully built dense-model-to-pipeline
  Attempting uninstall: dense-model-to-pipeline
    Found existing installation: dense-model-to-pipeline 1.0
    Uninstalling dense-model-to-pipeline-1.0:
      Successfully uninstalled dense-model-to-pipeline-1.0


Podemos agora realizar a importação do nosso pacote personalizado em nosso notabook!

Iremos importar a transformação DropColumns. Se você possui outras transformações personalizadas, não se esqueça de importá-las!

In [3]:
from my_custom_model.custom_dense_model_classifier import DenseModel
from my_custom_model.transforms import PrepareData

#### Declarando a Pipeline

Após a importação das transformações personalizadas como um pacote Python, podemos partir para a declaração da nossa Pipeline.

O processo é bem semelhante ao realizado na primeira etapa, porém com algumas diferenças importantes, então preste bem atenção!

A Pipeline exemplo possui três estágios: 

    - remover a coluna "NOME"
    - imputar "zeros" em todos os valores faltantes
    - inserir os dados pré-processados como entrada em um modelo treinado
    
Relembrando, a entrada desta Pipeline será o conjunto cru de dados fornecido exceto a coluna "PERFIL" (variável-alvo a ser determinada pelo modelo).

    MATRICULA       - número de matrícula do estudante
    NOME            - nome completo do estudante
    REPROVACOES_DE  - número de reprovações na disciplina de ``Direito Empresarial``
    REPROVACOES_EM  - número de reprovações na disciplina de ``Empreendedorismo``
    REPROVACOES_MF  - número de reprovações na disciplina de ``Matemática Financeira``
    REPROVACOES_GO  - número de reprovações na disciplina de ``Gestão Operacional``
    NOTA_DE         - média simples das notas do aluno na disciplina de ``Direito Empresarial`` (0-10)
    NOTA_EM         - média simples das notas do aluno na disciplina de ``Empreendedorismo`` (0-10)
    NOTA_MF         - média simples das notas do aluno na disciplina de ``Matemática Financeira`` (0-10)
    NOTA_GO         - média simples das notas do aluno na disciplina de ``Gestão Operacional`` (0-10)
    INGLES          - variável binária que indica se o estudante tem conhecimento em língua inglesa (0 -> sim ou 1 -> não).
    H_AULA_PRES     - horas de estudo presencial realizadas pelo estudante
    TAREFAS_ONLINE  - número de tarefas online entregues pelo estudante
    FALTAS          - número de faltas acumuladas do estudante (todas disciplinas)
    
A variável-alvo é:

PERFIL               - uma *string* que indica uma de cinco possibilidades: 
    "EXCELENTE"      - Estudante não necessita de mentoria
    "MUITO BOM"      - Estudante não necessita de mentoria
    "HUMANAS"        - Estudante necessita de mentoria exclusivamente em matérias com conteúdo de ciências humanas
    "EXATAS"         - Estudante necessita de mentoria apenas em disciplinas com conteúdo de ciências exatas
    "DIFICULDADE"    - Estudante necessita de mentoria em duas ou mais disciplinas

Com um modelo capaz de classificar um estudante em uma dessas categorias, podemos automatizar parte da mentoria estudantil através de assistentes virtuais, que serão capazes de recomendar práticas de estudo e conteúdo personalizado com base nas necessidades de cada aluno.

## ATENÇÃO: As "features" declaradas na célula abaixo são as entradas da PIPELINE, e não do seu modelo (as features da Pipeline serão TODAS as colunas do dataset exceto a coluna PERFIL, que é o nosso alvo).

In [4]:
# Definição das colunas que serão features (entradas) da Pipeline
features = [
    "MATRICULA", "NOME", 'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
    "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
    "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
]


In [5]:
# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_data_1[features]
Y = df_data_1["PERFIL"].values.ravel()

**** Preparação das transformações

In [6]:
drop_columns = ['NOME',"MATRICULA","FALTAS","INGLES"]
proporcional_in_columns = ["NOTA_DE","NOTA_GO","NOTA_MF","NOTA_EM"]
merge_comlumns_to_name = ["H_AULA_PRES","TAREFAS_ONLINE","ATIVIDADES"]
features_to_model = [
    'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
    "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO", "ATIVIDADES"
]
prepare_data = PrepareData(drop_columns,proporcional_in_columns,merge_comlumns_to_name)

**** Divisão dos dados

In [7]:
#X_train e Y_train serão dividos dentro do modelo para extrair X_val e Y_val, por isso test_size é pequeno aqui
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=337)

# Modelo

In [8]:
#input_s,num_classes, batch_size, epochs, activation, prepare_data
model = DenseModel(9,5,50,100,"relu",prepare_data)

Na célula abaixo é realizada a declaração de um objeto **Pipeline** do scikit-learn, onde é declarado o parâmetro *steps*, que nada mais é do que uma lista com as etapas da nossa pipeline:

    'prepare_data'     - transformação personalizad para preparação dos dados
    'model'         - Cria o modelo baseado em redes neurais com camadas totalmente conectadas
    
Note que passamos como passos as transformadas instanciadas anteriormente, sob nome `rm_columns` e `si`.

In [9]:
# Criação da nossa pipeline para armazenamento no Watson Machine Learning:
my_pipeline = Pipeline(
    steps=[
        ('prepare_data', prepare_data),
        ('model', model),
    ]
)

Em seguida iremos executar o método `fit()` da Pipeline, realizando o pré-processamento e o treinamento do modelo de uma só vez.

In [10]:
# Inicialização da Pipeline (pré-processamento e realização do treinamento do modelo)
my_pipeline.fit(X_train, y_train)

Epoch 1/50
144/144 [==============================] - 0s 3ms/step - loss: 1.1983 - accuracy: 0.5786 - val_loss: 1.0229 - val_accuracy: 0.6222
Epoch 2/50
144/144 [==============================] - 0s 2ms/step - loss: 0.9432 - accuracy: 0.6382 - val_loss: 0.8919 - val_accuracy: 0.6536
Epoch 3/50
144/144 [==============================] - 0s 2ms/step - loss: 0.8242 - accuracy: 0.6922 - val_loss: 0.7813 - val_accuracy: 0.7061
Epoch 4/50
144/144 [==============================] - 0s 2ms/step - loss: 0.7347 - accuracy: 0.7275 - val_loss: 0.7110 - val_accuracy: 0.7442
Epoch 5/50
144/144 [==============================] - 0s 2ms/step - loss: 0.6787 - accuracy: 0.7441 - val_loss: 0.6584 - val_accuracy: 0.7581
Epoch 6/50
144/144 [==============================] - 0s 2ms/step - loss: 0.6353 - accuracy: 0.7576 - val_loss: 0.6231 - val_accuracy: 0.7589
Epoch 7/50
144/144 [==============================] - 0s 2ms/step - loss: 0.6021 - accuracy: 0.7681 - val_loss: 0.5979 - val_accuracy: 0.7625
Epoch 

Pipeline(steps=[('prepare_data',
                 PrepareData(drop_colums=['NOME', 'MATRICULA', 'FALTAS',
                                          'INGLES'],
                             merge_comlumns_to_name=['H_AULA_PRES',
                                                     'TAREFAS_ONLINE',
                                                     'ATIVIDADES'],
                             proporcional_in_columns=['NOTA_DE', 'NOTA_GO',
                                                      'NOTA_MF', 'NOTA_EM'])),
                ('model',
                 DenseModel(activation='relu', batch_size=50, epochs=100,
                            input_s=9, num_classes=5,
                            prepare_data=PrepareData(drop_colums=['NOME',
                                                                  'MATRICULA',
                                                                  'FALTAS',
                                                                  'INGLES'],
                   

In [16]:
c = ["MATRICULA","NOME"]

X_train[c[0]]


6820     982644
17134    528843
4445     249367
17078    362984
15996    801090
          ...  
1087     677715
4539     271501
12729    625738
14402    443502
12548    201568
Name: MATRICULA, Length: 18000, dtype: int64

Agora que temos uma pipeline completa, com etapas de pré-processamento configuradas e também um modelo por árvore de decisão já treinado, podemos realizar a integração com o Watson Machine Learning!

<hr>

### Encapsulando uma Pipeline personalizada no Watson Machine Learning

#### Estabelecendo conexão entre o cliente Python do WML e a sua instância do serviço na nuvem

In [1]:
# Biblioteca Python com implementação de um cliente HTTP para a API do WML
from watson_machine_learning_client import WatsonMachineLearningAPIClient

ImportError: cannot import name 'joblib'

As próximas células irão realizar o deploy da pipeline declarada neste notebook no WML. Só prossiga se você já está satisfeito com seu modelo e acha que já é a hora de fazer o deploy da sua solução.

Cole as credenciais de sua instância do Watson Machine Learning na variável na célula abaixo.

É importante que a variável que contém os valores tenha o nome de ``wml_credentials`` para que as próximas células deste notebook executem corretamente.

In [ ]:
wml_credentials = {
  "apikey": "",
  "iam_apikey_description": "",
  "iam_apikey_name": "",
  "iam_role_crn": "",
  "iam_serviceid_crn": "",
  "instance_id": "",
  "url": ""
}

In [ ]:
# Instanciando um objeto cliente do Watson Machine Learning a partir das credenciais fornecidas

clientWML = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
# Extraindo detalhes da sua instância do Watson Machine Learning

instance_details = clientWML.service_instance.get_details()
print(json.dumps(instance_details, indent=4))

**ATENÇÃO!!**

Fique atento para os limites de consumo de sua instância do Watson Machine Learning!

Caso você expire a camada grátis, não será possível avaliar seu modelo (pois é necessária a realização de algumas chamadas de API que consomem predições!)

#### Listando todos os artefatos armazenados no seu WML

Para listar todos os artefatos armazenados em seu Watson Machine Learning, você pode usar a seguinte função:

    clientWML.repository.list()

In [ ]:
# Listando todos os artefatos atualmente armazenados na sua instância do WML

clientWML.repository.list()

No plano LITE do Watson Machine Learning é permitido um número limitado de artefatos. Se for o caso de você já possuir um modelo online na sua instância, você pode apagá-lo utilizando o método clientWML.repository.delete():

    artifact_guid = "359c8951-d2fe-4063-8706-cc06b32d5e0d"
    clientWML.repository.delete(artifact_guid)

#### Criando uma nova definição de pacote Python personalizado no WML

O primeiro passo para realizar seu deploy é armazenar o código das transformações personalizadas criadas por você.

Para essa etapa precisamos apenas do arquivo .zip do pacote criado (que já possuimos carregado no Kernel!)

In [ ]:
# Definição de metadados do nosso pacote com as Transforms personalizadas
pkg_meta = {
    clientWML.runtimes.LibraryMetaNames.NAME: "my_custom_sklearn_transform_1",
    clientWML.runtimes.LibraryMetaNames.DESCRIPTION: "A custom sklearn transform",
    clientWML.runtimes.LibraryMetaNames.FILEPATH: "sklearn_transforms.zip",  # Note que estamos utilizando o .zip criado anteriormente!
    clientWML.runtimes.LibraryMetaNames.VERSION: "1.0",
    clientWML.runtimes.LibraryMetaNames.PLATFORM: { "name": "python", "versions": ["3.6"] }
}
custom_package_details = clientWML.runtimes.store_library( pkg_meta )
custom_package_uid = clientWML.runtimes.get_library_uid( custom_package_details )

print("\n Lista de artefatos de runtime armazenados no WML:")
clientWML.repository.list()

#### Criando uma nova definição de runtime Python personalizado no WML

O segundo passo é armazenar uma definição de runtime Python para utilizar a nossa biblioteca personalizada.

Isso pode ser feito da seguinte forma:

In [ ]:
runtime_meta = {
    clientWML.runtimes.ConfigurationMetaNames.NAME: "my_custom_wml_runtime_1",
    clientWML.runtimes.ConfigurationMetaNames.DESCRIPTION: "A Python runtime with custom sklearn Transforms",
    clientWML.runtimes.ConfigurationMetaNames.PLATFORM: {
        "name": "python",
        "version": "3.6"
    },
    clientWML.runtimes.ConfigurationMetaNames.LIBRARIES_UIDS: [ custom_package_uid ]
}
runtime_details = clientWML.runtimes.store( runtime_meta )
custom_runtime_uid = clientWML.runtimes.get_uid( runtime_details )

print("\n Detalhes do runtime armazenado:")
print(json.dumps(runtime_details, indent=4))

In [ ]:
# Listando todos runtimes armazenados no seu WML:
clientWML.runtimes.list()

#### Criando uma nova definição de Pipeline personalizada no WML

Finalmente iremos criar uma definição (metadados) para a nossa Pipeline ser hospedada no WML.

Definimos como parâmetros um nome para o artefato e o ID do runtime criado anteriormente.

In [ ]:
model_meta = {
    clientWML.repository.ModelMetaNames.NAME: 'desafio-2-mbtc2020-pipeline-1',
    clientWML.repository.ModelMetaNames.DESCRIPTION: "my pipeline for submission",
    clientWML.repository.ModelMetaNames.RUNTIME_UID: custom_runtime_uid
}

Em seguida chamamos o método para armazenar a nova definição:

In [ ]:
# Função para armazenar uma definição de Pipeline no WML
stored_model_details = clientWML.repository.store_model(
    model=my_pipeline,  # `my_pipeline` é a variável criada anteriormente e contém nossa Pipeline já treinada :)
    meta_props=model_meta,  # Metadados definidos na célula anterior
    training_data=None  # Não altere esse parâmetro
)

print("\n Lista de artefatos armazenados no WML:")
clientWML.repository.list()

# Detalhes do modelo hospedado no Watson Machine Learning
print("\n Metadados do modelo armazenado:")
print(json.dumps(stored_model_details, indent=4))

#### Realizando o deployment do seu modelo para consumo imediato por outras aplicações

In [ ]:
# O deployment do modelo é finalmente realizado por meio do método ``deployments.create()``

model_deployment_details = clientWML.deployments.create(
    artifact_uid=stored_model_details["metadata"]["guid"],  # Não altere esse parâmetro
    name="desafio-2-mbtc2020-deployment-1",
    description="Solução do desafio 2 - MBTC",
    asynchronous=False,  # Não altere esse parâmetro
    deployment_type='online',  # Não altere esse parâmetro
    deployment_format='Core ML',  # Não altere esse parâmetro
    meta_props=model_meta  # Não altere esse parâmetro
)

#### Testando um modelo hospedado no Watson Machine Learning

In [ ]:
# Recuperando a URL endpoint do modelo hospedado na célula anterior

model_endpoint_url = clientWML.deployments.get_scoring_url(model_deployment_details)
print("A URL de chamada da sua API é: {}".format(model_endpoint_url))

### ATENÇÃO: Você irá precisar da URL acima para submeter seu modelo :)

In [ ]:
# Detalhes do deployment realizado

deployment_details = clientWML.deployments.get_details(
    deployment_uid=model_deployment_details["metadata"]["guid"]  # esse é o ID do seu deployment!
)

print("Metadados do deployment realizado: \n")
print(json.dumps(deployment_details, indent=4))

### Realizando uma chamada de API para seu modelo armazenado no WML

In [ ]:
scoring_payload = {
    'fields': [
        "MATRICULA", "NOME", 'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
        "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
        "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
    ],
    'values': [
        [
            513949,"Marli Quésia de Oliveira",1,1,1,1,4.3,4.0,3.1,4.9,0,3,4,3,
        ]
    ]
}

print("\n Payload de dados a ser classificada:")
print(json.dumps(scoring_payload, indent=4))

In [ ]:
result = clientWML.deployments.score(
    model_endpoint_url,
    scoring_payload
)

print("\n Resultados:")
print(json.dumps(result, indent=4))

<hr>

## Parabéns! 

Se tudo foi executado sem erros, você já tem um classificador baseado em machine learning encapsulado como uma API REST!

Para testar a sua solução integrada com um assistente virtual e realizar a submissão, acesse a página:

# https://uninassau.maratona.dev

Você irá precisar da endpoint url do seu modelo e das credenciais do WML :)

In [32]:
a = pd.DataFrame(X_test)

,MATRICULA,NOME,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,H_AULA_PRES,TAREFAS_ONLINE,FALTAS
5414,542158,Isabel Carolina de Padilha,0,0,0,0,5.9,5.4,4.6,4.9,1.0,0,1,6
7475,227579,Roger Gustavo Bahia de Leite,0,0,1,1,7.1,6.8,0.0,0.0,NaN,5,6,2
17254,375737,Leila Ezry,1,1,3,1,0.0,0.0,0.0,0.0,NaN,2,2,3
1716,711840,Rodrigo Saulo Prates da Silva,0,0,0,0,6.6,6.7,5.1,NaN,1.0,3,2,6
18538,451934,Otávio Ijaolman Osveorniman Júnior,0,0,0,0,5.8,6.1,5.4,5.0,0.0,9,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6219,913693,Ney Manuel de Bragança Quiazi,0,0,0,0,6.5,6.1,5.6,NaN,0.0,3,5,3
16757,994645,Ana de Madureira,0,0,0,0,5.4,4.8,4.5,NaN,NaN,5,5,5
3700,627766,Cibele Mikaella da Silva,1,1,1,1,0.0,0.0,0.0,NaN,1.0,7,4,5
1996,446708,Wanda Pabamo Crornasfulã,0,0,0,0,6.4,6.4,4.6,6.6,1.0,3,0,3
